# Практика: word2vec для кристаллов

Как вы уже могли заметить, идея, лежащая в основе [word2vec](https://arxiv.org/pdf/1310.4546), достаточно общая. В данном задании вы реализуете его самостоятельно.

P.s. GPU в этом задании нам не потребуется :)

Многие функции из этого ноутбука лежат в файле `skipgram_model_functions.py`. Это сделано специально, чтобы подсветить важные моменты и не отвлекаться на большое количество кода. Мы рекомендуем ничего не менять в функциях, исключением могут быть функции для визуализации данных.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import umap
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import PunktSentenceTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

from skipgram_model_functions import *

/home/u7c/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Что мы хотим сделать?

Это задание нужно для того, чтобы своими руками написать модель, которая является отправной точкой для современных LLM. Прежде чем углубляться в более сложные архитектуры и изучать то, как LLM на лету генерирует текст (а иногда и не текст, а что-то другой модальности), что можно считать работой с последовательностями динамической длины, мы хотим разобраться с построением контекстуального эмбеддинга для токена - атомарной единицы языка. Мы будем решать задачу предсказания вероятности одного токена оказаться (или не оказаться) в одном контексте с другим. Из этого знания мы можем получить представление о том, какие токены схожи и различны. Опираясь на эту информацию можно не только приступать к генерации последовательностей, но и решать другие задачи. Например, в кристаллохимии можно предсказывать физикохимические свойства соединений, зная их векторное представление, или эмбеддинг.

## Небольшой рассказ о данных

Мы будем работать с базой данных `Materials Project`. Для задачи были отобраны стабильные экспериментальные кристаллические структуры, для которых было сгенерировано текстовое описние при помощи `Robocrystallographer`. Все текстовые описания были токенизированы (в нашем случае токен = 1 слово). В `корпус` попали те структуры, описание которых не превышало 70 токенов. 
<p>Папка содержит 2 файла с данными:

`robocrys_exp_stable_70_tokens.txt` - основной файл, с которым мы будем работать. Содержит строки с текстовым описанием кристаллических структур.
<p>

`robocrys_exp_stable_70_tokens.csv` - файл, содержащий дополнительную информацию о кристаллических структурах, которая пригодится при визуализации эмбеддингов, которые мы посчитаем

In [2]:
data = list(open("robocrys_exp_stable_70_tokens.txt", encoding="utf-8"))

## Предобработка данных. Токенизация

Токенизация – первый шаг.
Тексты, с которыми мы работаем, включают в себя пунктуацию и прочие нестандартные токены, так что простой `str.split` иногда может не подходить под конкретную задачу. В данном случае в качестве токена мы будем использовать каждое слово, число и символ пространнственной группы. Можно было бы для каждой строки `i` в `data` выполнить `data[i].strip().split()`, но мы используем `WhitespaceTokenizer`, выполняющий аналогичную функцию.

Обратимся к `nltk` - библиотеке, которая нашла широкое применение в области NLP.

In [3]:
tokenizer = WhitespaceTokenizer()

Посмотрим, как выглядит типичное описание одного кристалла:

In [4]:
test_description = data[0]

print(test_description)

Hf is Magnesium structured and crystallizes in the hexagonal P63/mmc space group. Hf is bonded to twelve equivalent Hf atoms to form a mixture of edge, face, and corner-sharing HfHf12 cuboctahedra. There are six shorter (3.12 Å) and six longer (3.19 Å) Hf-Hf bond lengths.



Можно заметить, что описание содержит переносы строки, запятые и точки, которые необходимо удалить перед применением `WhitespaceTokenizer`. Все точки удалять нельзя, потому что некоторые из них являются частью чисел с плавающей точкой (например, 3.26 Å). Также нельзя удалять и использовать в качестве разделителей для токенизации некоторые специальные символы: /, -.

Сначала заменим все запятые в описаниях кристаллов на пробелы:

In [5]:
test_description = test_description.replace(",", "")

print(test_description)

Hf is Magnesium structured and crystallizes in the hexagonal P63/mmc space group. Hf is bonded to twelve equivalent Hf atoms to form a mixture of edge face and corner-sharing HfHf12 cuboctahedra. There are six shorter (3.12 Å) and six longer (3.19 Å) Hf-Hf bond lengths.



Теперь необходимо избавиться от символов переноса строки и точек в конце каждого предложения. Для этого используем `PunktSentenceTokenizer`. Он разобьёт описания на предложения, а затем мы удалим точку в конце каждого из них.

In [6]:
sentence_tokenizer = PunktSentenceTokenizer()

In [7]:
test_sentences = sentence_tokenizer.tokenize(test_description)

print(f'Test sentences in description look like:\n{test_sentences}')


# Вот теперь используем WhiteSpaceTokenizer
sentences_modified = [s.rstrip('.') for s in test_sentences]
test_tokens = sum([tokenizer.tokenize(sentence) for sentence in sentences_modified], [])

print(f'Test tokens of description look like:\n{test_tokens}')

Test sentences in description look like:
['Hf is Magnesium structured and crystallizes in the hexagonal P63/mmc space group.', 'Hf is bonded to twelve equivalent Hf atoms to form a mixture of edge face and corner-sharing HfHf12 cuboctahedra.', 'There are six shorter (3.12 Å) and six longer (3.19 Å) Hf-Hf bond lengths.']
Test tokens of description look like:
['Hf', 'is', 'Magnesium', 'structured', 'and', 'crystallizes', 'in', 'the', 'hexagonal', 'P63/mmc', 'space', 'group', 'Hf', 'is', 'bonded', 'to', 'twelve', 'equivalent', 'Hf', 'atoms', 'to', 'form', 'a', 'mixture', 'of', 'edge', 'face', 'and', 'corner-sharing', 'HfHf12', 'cuboctahedra', 'There', 'are', 'six', 'shorter', '(3.12', 'Å)', 'and', 'six', 'longer', '(3.19', 'Å)', 'Hf-Hf', 'bond', 'lengths']


Теперь токенизируем все описания кристаллов в нашем корпусе:

In [8]:
data_tokenized = []

for description in data:
    description = description.replace(",", "")
    sentences = sentence_tokenizer.tokenize(description)
    sentences_modified = [s.rstrip('.') for s in sentences]
    tokens = sum([tokenizer.tokenize(s) for s in sentences_modified], [])
    data_tokenized.append(tokens)


print(f"Количество описаний кристаллов в корпусе:\n{len(data_tokenized)}\n")

n_tokens = sum(len(description) for description in data_tokenized)
print(f"Суммарное количество токенов в корпусе:\n{n_tokens}")

Количество описаний кристаллов в корпусе:
1798

Суммарное количество токенов в корпусе:
104398


## Частота встречаемости слов и уникальные слова

Для расчета вероятностей нахождения одного слова в контексте другого нам необходимо получить частоту встречаемости каждого токена и список уникальных токенов. Сначала подсчитаем частоту встречаемости слов в корпусе:

In [9]:
word_count_dict = get_word_count_dict(data_tokenized)

print(f'Частота встречаемости каждого токена в корпусе:\n{word_count_dict}')

Частота встречаемости каждого токена в корпусе:
{'Hf': 49, 'is': 5182, 'Magnesium': 20, 'structured': 1095, 'and': 3336, 'crystallizes': 1798, 'in': 4524, 'the': 1798, 'hexagonal': 236, 'P63/mmc': 76, 'space': 1798, 'group': 1798, 'bonded': 3931, 'to': 5136, 'twelve': 388, 'equivalent': 4605, 'atoms': 3759, 'form': 1205, 'a': 4117, 'mixture': 1017, 'of': 1436, 'edge': 536, 'face': 223, 'corner-sharing': 1055, 'HfHf12': 1, 'cuboctahedra': 352, 'There': 677, 'are': 3016, 'six': 1202, 'shorter': 305, '(3.12': 4, 'Å)': 610, 'longer': 305, '(3.19': 5, 'Hf-Hf': 1, 'bond': 2679, 'lengths': 2204, 'Sb': 9, 'alpha': 26, 'As': 17, 'trigonal': 172, 'R̅3m': 22, 'distorted': 475, 'corner': 470, 'edge-sharing': 460, 'SbSb6': 1, 'octahedra': 901, 'The': 600, 'octahedral': 107, 'tilt': 96, 'angles': 96, '12°': 1, 'three': 373, '(2.93': 6, '(3.43': 6, 'Sb-Sb': 5, 'Y': 78, 'face-sharing': 172, 'YY12': 1, '(3.61': 1, '(3.64': 3, 'Y-Y': 1, 'Tc': 10, 'TcTc12': 1, '(2.71': 2, '(2.74': 5, 'Tc-Tc': 2, 'Zr': 57

Получим множество уникальных токенов. Это множество будет нашим `словарём`

In [10]:
vocabulary = set(word_count_dict.keys())

print(f'Уникальные токены:\n{vocabulary}\n')

print(f'Количество уникальных токенов в корпусе:\n{len(vocabulary)}')

Уникальные токены:
{'Nd-Te', 'HfIr12', 'P-I', 'LuPt', 'ErPt12', 'Na2LiAu3', 'Li2Pt', 'OsP6', 'OBa2Au2', 'Tm-B', 'Pnma', 'PrGe', 'cuboctahedra', '2.89', 'Pb2Rh', 'Os2-', 'TcAl12', 'CV6', 'GdAu', 'PrP', 'Eu-Ag', 'Na2CuP', 'PtTb6Pt6', 'ClNa6', 'HoSe6', 'Ga3+', 'ReB2', 'RbHg2', 'GdIr3', 'Nb-S', 'Au2Zr', 'Fe+2.50+', 'Li(RhSi)2', 'LuB12', 'CeN6', '2.78', '(2.85', 'Eu-P', 'SnO6', 'Sm2+', 'TbPd12', 'Dy-B', 'SrGe', 'LaPt', 'AlCa6Al6', 'TmNiBi', 'PLa6', 'Dy-Pt', 'TbPd', '2.55-2.71', 'Er-Sb', 'CrAu12', 'ILi4', 'TeCe6', 'LaSn3', 'HoH2', 'NbB', 'Sr-Ge', 'KAuF4', 'Sn-O', 'Mg-Mg', 'YGe', 'SbTi12', 'MgS', 'Tl3SbS3', 'Ce-Ge', 'Tb-Pt', 'ThSi2', 'ThSe6', 'K-H', '(3.17', '1.88', 'Na-I', 'NbRu2Al', 'LuRh', '(3.26', '2.47-2.69', 'MnGe', 'TeZr6', 'HgY', 'Co-Ga', 'IrDy6Ir6', 'Pa', 'Zintl', '2.02', 'MgHg', 'Li-Al', '2.24', '2.25', 'NbAu12', 'NaTl4', 'NdPt2', 'BeSe4', '2.82', 'LaPd', '2.92', 'Zr-B', 'Th-Ru', 'NbIr8', 'K2AgP', 'Pr-S', 'Au2GdIn', 'LaB6', 'Sm-Os', 'NpCl4', 'EuS6', '2.96-3.13', 'RuTa', 'Y1+', 'BiCe

Индексируем наш словарь:

In [11]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {index: word for word, index in word_to_index.items()}

## Создание пар `(слово, контекст)`

Для построения пар `(слово, контекст)` нам необходимо пройтись скользящим окном по всем описаниям в нашем корпусе.

Ниже задана ширина окна контекста Skipgram модели.

In [12]:
window_radius = 9

Пары `(слово, контекст)` на основе доступного датасета сгенерированы ниже.

In [13]:
context_pairs = get_context_pairs(data_tokenized, word_to_index, window_radius)

print(f"Количество пар (слово, контекст): {len(context_pairs)}")

Количество пар (слово, контекст): 1629128


## Подзадача №1: subsampling
Для того, чтобы сгладить разницу в частоте встречаемсости слов, необходимо реализовать механизм subsampling'а.
Для этого вам необходимо реализовать функцию ниже.

Вероятность **исключить** слово из обучения (на фиксированном шаге) вычисляется как
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
где $f(w_i)$ – нормированная частота встречаемости слова, а $t$ – заданный порог (threshold).

In [14]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """

    words_count = sum(word_count_dict.values())

    keep_prob_dict = {}  

    for word, count in word_count_dict.items():

        freq_norm = count / words_count
        include_prob = (threshold / freq_norm) ** 0.5 if freq_norm > threshold else 0
        keep_prob_dict[word] = include_prob
        
    return keep_prob_dict

## Подзадача №2: negative sampling
Для более эффективного обучения необходимо не только предсказывать высокие вероятности для слов из контекста, но и предсказывать низкие для слов, не встреченных в контексте. Для этого вам необходимо вычислить вероятность использовать слово в качестве negative sample, реализовав функцию ниже.

`ВНИМАНИЕ: этот код вам нужно написать самостоятельно`

В оригинальной статье предлагается оценивать вероятность слов выступать в качестве negative sample согласно распределению $P_n(w)$
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

где $U(w)$ распределение слов по частоте (или, как его еще называют, по униграммам), а $Z$ – нормировочная константа, чтобы общая мера была равна $1$.

`ПОДСКАЗКА`: распределение слов по частоте есть ничто иное как частота встречаемости слова в корпусе

In [15]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """

    ### YOUR CODE HERE

    adjusted_freqs = {word: count ** (3/4) for word, count in word_count_dict.items()}
    Z = sum(adjusted_freqs.values())
    negative_sampling_prob_dict = {word: freq / Z for word, freq in adjusted_freqs.items()}

    return negative_sampling_prob_dict

Применим функции `subsample_frequent_words` для расчета вероятностей слов быть отобранными в обучающую выборку и `get_negative_sampling_prob` для расчета вероятностей слов быть отобранными в выборку negative_sampling.<p>
Для удобства, преобразуем полученные словари в массивы (т.к. все слова все равно уже пронумерованы).

In [16]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

keep_prob_array = keep_prob_to_array(keep_prob_dict, index_to_word, word_to_index)

In [17]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

negative_sampling_prob_array = negative_sampling_prob_to_array(negative_sampling_prob_dict, index_to_word, word_to_index)

### Реализация модели Skipgram

Наконец, время реализовать модель!

Напомним, что в случае negative sampling решается задача максимизации следующего функционала:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

где:
- $\mathbf{v}_{w_I}$ – вектор центрального слова $w_I$,
- $\mathbf{v}'_{w_O}$ – вектор слова из контекста $w_O$,
- $k$ – число negative samples,
- $P_n(w)$ – распределение negative samples, заданное выше,
- $\sigma$ – сигмоида.

In [18]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center_words, pos_context_words, neg_context_words):

        center_embeddings = self.center_embeddings(center_words)
        pos_context_embeddings = self.context_embeddings(pos_context_words)
        neg_context_embeddings = self.context_embeddings(neg_context_words)

        pos_scores = torch.sum(center_embeddings * pos_context_embeddings, dim=1)
        neg_scores = torch.sum((center_embeddings.unsqueeze(1) * neg_context_embeddings), dim = 2)

        return pos_scores, neg_scores

## Обучаем модель

Инициализируем модель. Не забываем инициализировать её перед каждым обучением!

In [19]:
device = torch.device("cpu")

vocab_size = len(word_to_index)
embedding_dim = 32
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)
criterion = nn.BCEWithLogitsLoss()

params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

Запускаем обучение:

In [ ]:
steps = 2500
batch_size = 1024

train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
)

Сохраняем состояние обученной модельки:

In [24]:
#torch.save(model.state_dict(), 'my_model_100_test.pth')

## Тестируем модель

Оценим Accuracy модели:

In [21]:
accuracy = evaluate_model(model, context_pairs, device)

Evaluating: 100%|██████████| 25456/25456 [00:02<00:00, 11601.96it/s]


Accuracy: 0.6834


Наконец, взглянем на ближайшие по косинусной мере слова:

In [22]:
central_word = "He"
n_neighbours = 20

find_nearest(model, central_word, word_to_index, index_to_word, n_neighbours)

[('RhHo6Rh6', 0.47442108392715454),
 ('C4+', 0.4749743342399597),
 ('Å)', 0.4750446081161499),
 ('YRu2Si2', 0.4798634648323059),
 ('(3.14', 0.4859497547149658),
 ('Rb(BH)6', 0.48756512999534607),
 ('La-Os', 0.48966145515441895),
 ('Co2TiAl', 0.4932682514190674),
 ('3.50', 0.49957871437072754),
 ('GdPd', 0.4997093081474304),
 ('KBrF4', 0.4998997449874878),
 ('MgMg12', 0.5043226480484009),
 ('V-As', 0.507789671421051),
 ('PDy6', 0.5102002024650574),
 ('ZnSe', 0.5113644599914551),
 ('47°', 0.5219210982322693),
 ('HoS', 0.5503510236740112),
 ('N3+', 0.5535392165184021),
 ('(2.20', 0.5754271149635315),
 ('He', 0.9999998807907104)]

## Визуализация результатов

Для начала нам необходимо из всего списка слов отобрать те, которые мы хотим визуализировать. 
<p>Мы хотим визуализировать формулы кристаллов, отберем их: 

In [23]:
list_of_formules = [description[0] for description in data_tokenized]

Получаем эмбеддинги для отобранных слов:

In [24]:
word_embeddings = get_word_embeddings(model, word_to_index, list_of_formules)

Наши эмбеддинги имеют размерность 32, а мы хотим визуализировать их на интерактивном 3D-графике. Для этого необходимо понизить размерность эмбеддингов с помощью редьюсеров. Мы будем использовать `umap`.

In [25]:
umap_model = umap.UMAP(n_components=3, n_neighbors=5)

embeddings_3d = umap_model.fit_transform(word_embeddings)

Строим интерактивные 3D-графики.
Ниже вы можете ввести название файла, в который будет сохранен интерактивный график. Открыть его можно на локальном компьютере в браузере. Все остальные переменные лучше не трогать, но если хочется настроить график под себя, можно исправить функции в файле `skipgram_model_functions.py`

In [37]:
output_filename = "3D_plot_by_structure_types_70_tokens.html"

properties_filename = 'robocrys_exp_stable_70_tokens.csv'
get_3D_plot_by_structure_types(list_of_formules, embeddings_3d, accuracy, output_filename, properties_filename)

In [39]:
output_filename = "3D_plot_by_crystal_system_70_tokens.html"

properties_filename = 'robocrys_exp_stable_70_tokens.csv'
get_3D_plot_by_crystal_system(list_of_formules, embeddings_3d, accuracy, output_filename, properties_filename)

### Над чем стоит подумать?

<li> Как повысить accuracy?
<li> Возможно, вы заметили в токенизации некоторые баги. Подумайте, с помощью каких инструментов от них удобнее всего избавиться.
<li> Как ширина контекстного окна Skipgram модели влияет на результат? Что изменится, если его уменьшить/увеличить?
<li> Если получилось отметить явно выраженные кластеры на 3D-графиках, подумать, какая закономерность в том, что эмбединги структур, находящихся в кластере, схожи.